In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [6]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 168 ( 0.5%) Users that have less than 1 test interactions


In [7]:
from libs.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

In [9]:
ItemKNNCFRecommender(URM_train).fit(similarity="asymmetric")

Similarity column 38121 (100.0%), 4609.88 column/sec. Elapsed time 8.27 sec


In [12]:
def objective_function_KNN(optuna_trial):
                       
        
    recommender_instance = ItemKNNCFRecommender(URM_train)
    
    similarity = 'euclidean'
    
    full_hyperp = {"similarity": similarity,
                   "topK": optuna_trial.suggest_int("topK", 2, 100),
                   "shrink": optuna_trial.suggest_int("shrink", 1 , 50),
                   "feature_weighting": optuna_trial.suggest_categorical("feature_weighting", ["BM25", "TF-IDF", "none"])
                  }
    
    if similarity == "asymmetric":
        full_hyperp["asymmetric_alpha"] = optuna_trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        full_hyperp["normalize"] = True

    elif similarity == "tversky":
        full_hyperp["tversky_alpha"] = optuna_trial.suggest_float("tversky_alpha", 0.1, 0.99, log=False)
        full_hyperp["tversky_beta"] = optuna_trial.suggest_float("tversky_beta", 0.1, 1.9, log=False)
        full_hyperp["normalize"] = True 

    elif similarity == "euclidean":
        full_hyperp["normalize_avg_row"] = optuna_trial.suggest_categorical("normalize_avg_row", [True, False])
        full_hyperp["similarity_from_distance_mode"] = optuna_trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        full_hyperp["normalize"] = optuna_trial.suggest_categorical("normalize", [True, False])
        
    
    recommender_instance.fit(**full_hyperp)
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)    
    return eval_res["MAP"][10]

In [13]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame()
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        
        self.results_df = self.results_df._append(hyperparam_dict, ignore_index=True)

In [15]:
optuna_study = op.create_study(direction="maximize")
        
save_results_euclidian = SaveResults()
        
optuna_study.optimize(objective_function_KNN,
                      callbacks=[save_results_euclidian],
                      n_trials = 250)

[I 2024-12-11 00:28:40,380] A new study created in memory with name: no-name-efc75200-8ce1-4db6-80a8-58524fe83bcd


Similarity column 38121 (100.0%), 1178.91 column/sec. Elapsed time 32.34 sec
EvaluatorHoldout: Processed 35568 (100.0%) in 11.31 sec. Users per second: 3145


[I 2024-12-11 00:29:24,098] Trial 0 finished with value: 0.02592084065219966 and parameters: {'topK': 65, 'shrink': 3, 'feature_weighting': 'BM25', 'normalize_avg_row': True, 'similarity_from_distance_mode': 'log', 'normalize': True}. Best is trial 0 with value: 0.02592084065219966.


Similarity column 38121 (100.0%), 1075.67 column/sec. Elapsed time 35.44 sec
EvaluatorHoldout: Processed 35568 (100.0%) in 9.13 sec. Users per second: 3894


[I 2024-12-11 00:30:08,724] Trial 1 finished with value: 0.04713635495433576 and parameters: {'topK': 2, 'shrink': 24, 'feature_weighting': 'TF-IDF', 'normalize_avg_row': False, 'similarity_from_distance_mode': 'log', 'normalize': True}. Best is trial 1 with value: 0.04713635495433576.


...

[I 2024-12-11 03:22:19,360] Trial 247 finished with value: 0.04606818305557174 and parameters: {'topK': 6, 'shrink': 38, 'feature_weighting': 'TF-IDF', 'normalize_avg_row': False, 'similarity_from_distance_mode': 'log', 'normalize': True}. Best is trial 198 with value: 0.04831407110189834.


Similarity column 38121 (100.0%), 1255.17 column/sec. Elapsed time 30.37 sec
EvaluatorHoldout: Processed 35568 (100.0%) in 8.51 sec. Users per second: 4179


[I 2024-12-11 03:22:58,295] Trial 248 finished with value: 0.04791303958792832 and parameters: {'topK': 4, 'shrink': 23, 'feature_weighting': 'TF-IDF', 'normalize_avg_row': False, 'similarity_from_distance_mode': 'log', 'normalize': True}. Best is trial 198 with value: 0.04831407110189834.


Similarity column 38121 (100.0%), 1205.98 column/sec. Elapsed time 31.61 sec
EvaluatorHoldout: Processed 35568 (100.0%) in 8.31 sec. Users per second: 4279


[I 2024-12-11 03:23:38,270] Trial 249 finished with value: 0.04715704414169169 and parameters: {'topK': 2, 'shrink': 42, 'feature_weighting': 'TF-IDF', 'normalize_avg_row': True, 'similarity_from_distance_mode': 'exp', 'normalize': True}. Best is trial 198 with value: 0.04831407110189834.


In [11]:
save_results.results_df.to_csv("result_experiments/ItemKNNCFRecommender/results_asymmetric.csv")